In [21]:
import pandas as pd
import mysql.connector
from datetime import datetime

# Step 1: Load Data
df = pd.read_csv(r"F:\SQL\traffic_stops - traffic_stops_with_vehicle_number.csv",low_memory=False)

In [22]:
df.head()

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [23]:
df.isnull().sum()

stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64

In [24]:
#Step 2: Data Cleaning

# Drop columns where all values are null
df.dropna(axis=1, how='all',inplace=True)

# Drop unwanted raw columns
df = df.drop(columns=['driver_age_raw', 'violation_raw'])

# Handle Missing (NaN) Values
df.fillna({'search_type': 'None'}, inplace=True)


# Convert date and time columns
#df['stop_date'] = pd.to_datetime(df['stop_date'], format = '%Y-%m-%d')
#df['stop_time'] = pd.to_datetime(df['stop_time']).dt.strftime('%H:%M:%S')

# Merge stop_date and stop_time 
df['stop_datetime'] = pd.to_datetime(df['stop_date'] + " " + df['stop_time']) 

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   stop_date           65538 non-null  object        
 1   stop_time           65538 non-null  object        
 2   country_name        65538 non-null  object        
 3   driver_gender       65538 non-null  object        
 4   driver_age          65538 non-null  int64         
 5   driver_race         65538 non-null  object        
 6   violation           65538 non-null  object        
 7   search_conducted    65538 non-null  bool          
 8   search_type         65538 non-null  object        
 9   stop_outcome        65538 non-null  object        
 10  is_arrested         65538 non-null  bool          
 11  stop_duration       65538 non-null  object        
 12  drugs_related_stop  65538 non-null  bool          
 13  vehicle_number      65538 non-null  object    

In [26]:
# Step 3: Initialize Database Connection

import mysql.connector




mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",

)


print(mydb)
mycursor = mydb.cursor(buffered=True)

In [27]:
# Step 4: Create A Database

mycursor.execute("create database securecheck")

In [28]:
# Use SecureCheck Database

mycursor.execute("USE securecheck;")
mydb.commit()

In [29]:
# Step 5: Create a Table
mycursor.execute('''CREATE TABLE traffic_stops (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(50),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(50),
    is_arrested BOOLEAN,
    stop_duration VARCHAR(20),
    drugs_related_stop BOOLEAN,
    vehicle_number VARCHAR(20),
    stop_datetime DATETIME
);''')

In [30]:
# Step 6: Insert traffic_stops data

# Prepare insert query
insert_query = """
INSERT INTO traffic_stops (
    stop_date, stop_time, country_name, driver_gender,
    driver_age, driver_race, violation,
    search_conducted, search_type, stop_outcome,
    is_arrested, stop_duration, drugs_related_stop, vehicle_number, stop_datetime 
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from DataFrame
for _, row in df.iterrows():
    data = (
        row['stop_date'], row['stop_time'], row['country_name'],
        row['driver_gender'], row['driver_age'], row['driver_race'],
        row['violation'], bool(row['search_conducted']), row['search_type'],
        row['stop_outcome'], bool(row['is_arrested']),
        row['stop_duration'], bool(row['drugs_related_stop']), row['vehicle_number'], row['stop_datetime']
    )
    mycursor.execute(insert_query, data)
    mydb.commit()
